In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential


In [4]:
img_height = 416
img_width = 416
batch_size = 32

import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from typing import List, Dict

def load_annotations(json_file) -> List[Dict[str, any]]:
    with open(json_file, 'r') as f:
        annotations = json.load(f)
    return annotations

def apply_transformations(image_dir, annotations_file):
    annotations = load_annotations(annotations_file)
    datagen = ImageDataGenerator(
        rotation_range=10,  # rotate images by 10 degrees
        width_shift_range=0.1,  # shift images horizontally by 10%
        height_shift_range=0.1,  # shift images vertically by 10%
        shear_range=0.2,  # shear transformations
        zoom_range=0.2,  # zoom transformations
        horizontal_flip=True,  # flip images horizontally
        vertical_flip=False,  # do not flip images vertically
    )

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            image_path = os.path.join(image_dir, filename)
            image = tf.keras.preprocessing.image.load_img(image_path)
            image_array = tf.keras.preprocessing.image.img_to_array(image)
            image_array = tf.expand_dims(image_array, 0)  # Add batch dimension

            augmented_images = datagen.flow(image_array, batch_size=1)

            for i, augmented_image in enumerate(augmented_images):
                augmented_image = augmented_image[0].astype('uint8')
                new_filename = f"transformed_{i}_{filename}"
                new_image_path = os.path.join(image_dir, new_filename)
                tf.keras.preprocessing.image.save_img(new_image_path, augmented_image)

                # Update annotations for the transformed image (optional)
                # if filename in annotations["image"]:

                annotation = next(filter(lambda x: x["image"] == filename, annotations), None)
                if annotation is not None:
                    # TODO figure out how to apply the same transformation to the bounding boxes
                    pass

                    # transformed_annotations = annotations[filename]

                    # transformed_annotations['filename'] = new_filename
                    # transformed_annotations_path = os.path.join(image_dir, f"{new_filename}_annotations.createml.json")
                    # with open(transformed_annotations_path, 'w') as f:
                    #     json.dump(transformed_annotations, f)

                if i >= 9:  # Generate a maximum of 10 transformed images
                    break

# Usage example
training_dir = './train'
annotations_file = './train/_annotations.createml.json'
apply_transformations(training_dir, annotations_file)



Found 0 files belonging to 0 classes.


ValueError: No images found in directory train/. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')